In [209]:
import pandas as pd
import ast

In [210]:
df = pd.read_csv("eurocup_2020_results.csv")

In [211]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."
5,Quarter-finals,02.07.2021,False,False,False,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"[{'event_team': 'away', 'event_time': "" 13' "",...","[{'Player_Name': 'Lukaku', 'Player_Number': '9...","[{'Player_Name': 'Insigne', 'Player_Number': '..."
6,Quarter-finals,02.07.2021,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': "" 8' "", ...","[{'Player_Name': 'Seferovic', 'Player_Number':...","[{'Player_Name': 'Sarabia', 'Player_Number': '..."
7,Round of 16,29.06.2021,False,False,False,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"[{'event_team': 'away', 'event_time': "" 27' "",...","[{'Player_Name': 'Isak', 'Player_Number': '11'...","[{'Player_Name': 'Yaremchuk', 'Player_Number':..."
8,Round of 16,29.06.2021,False,False,False,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"[{'event_team': 'home', 'event_time': "" 8' "", ...","[{'Player_Name': 'Sterling', 'Player_Number': ...","[{'Player_Name': 'Werner', 'Player_Number': '1..."
9,Round of 16,28.06.2021,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': "" 15' "",...","[{'Player_Name': 'Mbappe', 'Player_Number': '1...","[{'Player_Name': 'Seferovic', 'Player_Number':..."


In [212]:
def read_d_data(element):
    try:
        return ast.literal_eval(element) 
    except:
        return False

In [213]:
def clean_percentages(num):            
    return int(num.split('%')[0])

In [214]:
def clean_spaces(name):
    return name.strip()

In [215]:
def clean_pen(data):
    if data == 'False':
        return False
    else:
        return int(data)

In [216]:
df["pens_home_score"] = df["pens_home_score"].apply(clean_pen) #Limpio los datos de los penales, ya que el False y el numero de penales era un STR, y al sumarlo 3+4 me daba 34 :'('
df["pens_away_score"] = df["pens_away_score"].apply(clean_pen)

In [217]:
df["team_name_home"]= df["team_name_home"].apply(clean_spaces) #Limpiando espacios en los nombres de los paises.
df["team_name_away"]= df["team_name_away"].apply(clean_spaces)

In [218]:
df["possession_away"] = df["possession_away"].apply(clean_percentages) #Limpiando el porcentaje, y convirtiendo en numero, ya que era un str.
df["possession_home"] = df["possession_home"].apply(clean_percentages)

In [219]:
df["lineup_home"] = df["lineup_home"].apply(read_d_data) #Convierto estos tres STR en listas.
df["lineup_away"] = df["lineup_away"].apply(read_d_data)
df["events_list"] = df["events_list"].apply(read_d_data)

In [220]:
teams = list(df['team_name_home'].unique()) #Creo una lista de paises
teams.sort()
teams

['Austria',
 'Belgium',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'England',
 'Finland',
 'France',
 'Germany',
 'Hungary',
 'Italy',
 'Netherlands',
 'North Macedonia',
 'Poland',
 'Portugal',
 'Russia',
 'Scotland',
 'Slovakia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'Wales']

In [221]:
def goals_scored(team):
    goals_home = df[(df['team_name_home'] == team)]['team_home_score'].sum()
    goals_away = df[(df['team_name_away'] == team)]['team_away_score'].sum()
    return int(goals_home + goals_away), team

In [222]:
goals_favor = []
for team in teams:
    goals_favor.append(goals_scored(team))
goals_favor 

[(5, 'Austria'),
 (9, 'Belgium'),
 (7, 'Croatia'),
 (6, 'Czech Republic'),
 (12, 'Denmark'),
 (11, 'England'),
 (1, 'Finland'),
 (7, 'France'),
 (6, 'Germany'),
 (3, 'Hungary'),
 (13, 'Italy'),
 (8, 'Netherlands'),
 (2, 'North Macedonia'),
 (4, 'Poland'),
 (7, 'Portugal'),
 (2, 'Russia'),
 (1, 'Scotland'),
 (2, 'Slovakia'),
 (13, 'Spain'),
 (5, 'Sweden'),
 (8, 'Switzerland'),
 (1, 'Turkey'),
 (6, 'Ukraine'),
 (3, 'Wales')]

In [223]:
def goals_against(team):
    against_home = df[(df['team_name_home'] == team)]['team_away_score'].sum()
    against_away = df[(df['team_name_away'] == team)]['team_home_score'].sum()
    return int(against_home + against_away), team

In [224]:
goals_received = []
for team in teams:
    goals_received.append(goals_against(team))
goals_received 

[(5, 'Austria'),
 (3, 'Belgium'),
 (8, 'Croatia'),
 (4, 'Czech Republic'),
 (7, 'Denmark'),
 (2, 'England'),
 (3, 'Finland'),
 (6, 'France'),
 (7, 'Germany'),
 (6, 'Hungary'),
 (4, 'Italy'),
 (4, 'Netherlands'),
 (8, 'North Macedonia'),
 (6, 'Poland'),
 (7, 'Portugal'),
 (7, 'Russia'),
 (5, 'Scotland'),
 (7, 'Slovakia'),
 (6, 'Spain'),
 (4, 'Sweden'),
 (9, 'Switzerland'),
 (8, 'Turkey'),
 (10, 'Ukraine'),
 (6, 'Wales')]

In [225]:
def possession_team(team):
    possession_home = df[df['team_name_home'] == team]['possession_home'].sum()
    possession_away = df[df['team_name_away'] == team]['possession_away'].sum()
    match_played =  df[(df['team_name_away'] == team)|(df['team_name_home'] == f'{team}')]['stage'].count()
    possesion_total = (possession_home+possession_away)/match_played
    return round(float(possesion_total), 1), team

In [226]:
possession_total_team = []
for team in teams:
    possession_total_team.append(possession_team(team))
possession_total_team 

[(53.5, 'Austria'),
 (53.8, 'Belgium'),
 (49.5, 'Croatia'),
 (48.2, 'Czech Republic'),
 (53.3, 'Denmark'),
 (51.0, 'England'),
 (36.7, 'Finland'),
 (53.2, 'France'),
 (62.8, 'Germany'),
 (29.0, 'Hungary'),
 (54.9, 'Italy'),
 (54.8, 'Netherlands'),
 (41.7, 'North Macedonia'),
 (49.3, 'Poland'),
 (54.2, 'Portugal'),
 (42.3, 'Russia'),
 (43.3, 'Scotland'),
 (45.0, 'Slovakia'),
 (73.5, 'Spain'),
 (33.2, 'Sweden'),
 (47.8, 'Switzerland'),
 (49.7, 'Turkey'),
 (49.0, 'Ukraine'),
 (37.0, 'Wales')]

In [227]:
def shots(team):
    shots_home = df[(df['team_name_home'] == team)]['total_shots_home'].sum()
    shots_away = df[(df['team_name_away'] == team)]['total_shots_away'].sum()
    return int(shots_home + shots_away), team

In [228]:
total_shots =[]
for team in teams:
    total_shots.append(shots(team))
total_shots

[(53, 'Austria'),
 (48, 'Belgium'),
 (42, 'Croatia'),
 (56, 'Czech Republic'),
 (92, 'Denmark'),
 (63, 'England'),
 (19, 'Finland'),
 (56, 'France'),
 (49, 'Germany'),
 (19, 'Hungary'),
 (127, 'Italy'),
 (52, 'Netherlands'),
 (33, 'North Macedonia'),
 (40, 'Poland'),
 (51, 'Portugal'),
 (25, 'Russia'),
 (41, 'Scotland'),
 (24, 'Slovakia'),
 (115, 'Spain'),
 (41, 'Sweden'),
 (67, 'Switzerland'),
 (40, 'Turkey'),
 (51, 'Ukraine'),
 (39, 'Wales')]

In [229]:
def pen(team):
    pen_home = (df[(df['team_name_home'] == team) & (df['pens_home_score'] != False)]['pens_home_score'].sum())
    pen_away = (df[(df['team_name_away'] == team) & (df['pens_away_score'] != False)]['pens_away_score'].sum())
    return (pen_home + pen_away),team

In [230]:
penaltys_total = []
for team in teams:
    penaltys_total.append(pen(team))

In [231]:
penaltys_total

[(0, 'Austria'),
 (0, 'Belgium'),
 (0, 'Croatia'),
 (0, 'Czech Republic'),
 (0, 'Denmark'),
 (2, 'England'),
 (0, 'Finland'),
 (4, 'France'),
 (0, 'Germany'),
 (0, 'Hungary'),
 (7, 'Italy'),
 (0, 'Netherlands'),
 (0, 'North Macedonia'),
 (0, 'Poland'),
 (0, 'Portugal'),
 (0, 'Russia'),
 (0, 'Scotland'),
 (0, 'Slovakia'),
 (5, 'Spain'),
 (0, 'Sweden'),
 (6, 'Switzerland'),
 (0, 'Turkey'),
 (0, 'Ukraine'),
 (0, 'Wales')]

In [171]:
#En cada función retorno el team, para luego ver visualmente que dato pertenece a cada pais, luego lo borraré.